## Load to Gold Table

* Data **enrichment**

In [0]:
%sql
-- Create Gold table and define a schema
CREATE OR REPLACE TABLE workspace.`breakable-toy`.social_media_engagement_gold (
  post_id STRING,
  platform STRING,
  post_timestamp TIMESTAMP,
  post_date DATE,
  post_time STRING,
  likes DOUBLE,
  comments DOUBLE,
  shares DOUBLE,
  reach DOUBLE,
  total_engagements DOUBLE,
  engagement_rate DOUBLE
);

In [0]:
%sql
-- Insert data from the source table (social_media_engagement_silver)
-- into the target table (social_media_engagement_gold)
-- with additional aggregates
MERGE INTO workspace.`breakable-toy`.social_media_engagement_gold
SELECT 
  post_id, 
  platform,
  post_timestamp, 
  to_date(date_format(post_timestamp, 'yyyy-MM-dd')) AS post_date, 
  date_format(post_timestamp, 'HH:mm:ss') AS post_time,
  likes, 
  shares, 
  comments, 
  reach,
  (COALESCE(likes, 0) + COALESCE(shares, 0) + COALESCE(comments, 0)) AS total_engagements,
  CASE 
        WHEN COALESCE(reach, 0) > 0 THEN 
            ((COALESCE(likes, 0) + COALESCE(shares, 0) + COALESCE(comments, 0)) / COALESCE(reach, 1)) * 100
        ELSE 0
    END AS engagement_rate
FROM workspace.`breakable-toy`.social_media_engagement_silver